In [5]:
from time import time
import unittest
import numpy as np
from sklearn.metrics import silhouette_score

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1' 

from keras.datasets import mnist
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.48
set_session(tf.Session(config=config))

import sys  
sys.path.append('../')
from util.util import *
from util import importNotebook
from trial.autoencoder import VAE, ConvVAE

In [6]:
numEpochs = 100
sizeBatch = 32
sizeKernel = 3
layerDense = [64, 16, 4]
ratRecon = 1
nameOptim = 'adam'
modelPath = '../model/temp/'
patience = 3

(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
numTrain = len(xTrain)
numTest = len(xTest)
numClass = 10
sizeDigit = xTrain.shape[1:]

dimInput = np.prod(xTrain.shape[1:])
xTrain = xTrain.reshape((numTrain, dimInput))
xTest = xTest.reshape((numTest, dimInput))
print(xTrain.shape)
print(xTest.shape)

vae = VAE(dimInput, layerDense=layerDense, ratRecon=ratRecon)
history, timeTrain = vae.fit(xTrain, xTest, 
                                  numEpochs=numEpochs,
                                  sizeBatch=sizeBatch,
                                  tempPathBest=modelPath)

(60000, 784)
(10000, 784)
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 10s 167us/step - loss: 0.2183 - val_loss: 0.1839

Epoch 00001: val_loss improved from inf to 0.18386, saving model to ../model/temp/Conv_AutoEncoder.01-0.22-0.18.hdf5
Epoch 2/100
60000/60000 [==============================] - 9s 158us/step - loss: 0.1770 - val_loss: 0.1724

Epoch 00002: val_loss improved from 0.18386 to 0.17238, saving model to ../model/temp/Conv_AutoEncoder.02-0.18-0.17.hdf5
Epoch 3/100
60000/60000 [==============================] - 10s 159us/step - loss: 0.1700 - val_loss: 0.1676

Epoch 00003: val_loss improved from 0.17238 to 0.16756, saving model to ../model/temp/Conv_AutoEncoder.03-0.17-0.17.hdf5
Epoch 4/100
60000/60000 [==============================] - 9s 154us/step - loss: 0.1659 - val_loss: 0.1642

Epoch 00004: val_loss improved from 0.16756 to 0.16424, saving model to ../model/temp/Conv_AutoEncoder.04-0.17-0.16.hdf5
Epoch 5/10

60000/60000 [==============================] - 9s 158us/step - loss: 0.1500 - val_loss: 0.1515

Epoch 00040: val_loss did not improve from 0.15098
Epoch 41/100
60000/60000 [==============================] - 9s 157us/step - loss: 0.1500 - val_loss: 0.1507

Epoch 00041: val_loss improved from 0.15098 to 0.15065, saving model to ../model/temp/Conv_AutoEncoder.41-0.15-0.15.hdf5
Epoch 42/100
60000/60000 [==============================] - 9s 157us/step - loss: 0.1499 - val_loss: 0.1507

Epoch 00042: val_loss did not improve from 0.15065
Epoch 43/100
60000/60000 [==============================] - 9s 155us/step - loss: 0.1498 - val_loss: 0.1511

Epoch 00043: val_loss did not improve from 0.15065
Epoch 44/100
60000/60000 [==============================] - 9s 157us/step - loss: 0.1497 - val_loss: 0.1508

Epoch 00044: val_loss did not improve from 0.15065
Epoch 00044: early stopping


In [7]:
encoder = convVAE.encoder
decoder = convVAE.decoder

# Get encoded and decoded values
encodeTest = encoder.predict(xTest)
decodeTest = decoder.predict(encodeTest)
meanEncTest = np.mean(encodeTest, axis=0)
stdEncTest = np.std(encodeTest, axis=0)

# Plot the comparison of original and reconstructed pictures, and calculate the errors
plotCompDecode(xTest, decodeTest, sizeDigit=sizeDigit)
err = compReconst(xTest, decodeTest, method='mse')
print(err)

# Plot the scatter of the encoding space
xlim = (meanEncTest[0] - 4*stdEncTest[0], meanEncTest[0] + 4*stdEncTest[0])
ylim = (meanEncTest[1] - 4*stdEncTest[1], meanEncTest[1] + 4*stdEncTest[1])
plotScatterEncode(encodeTest, yTest, xlim, ylim, numShow=10000)
scoreSilh = silhouette_score(encodeTest, yTest)    

# Plot the decoding results from the encoding scatter
plotScatterDecode(decoder, sizeDigit, xlim, ylim)

NameError: name 'convVAE' is not defined

In [ ]:
numEpochs = 100
sizeBatch = 32
sizeKernel = 3
layerDense = [64, 16, 4]
layerConv=[8, 32]
ratRecon = 1
nameOptim = 'adam'
modelPath = '../model/temp/'
patience = 3

(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
numTrain = len(xTrain)
numTest = len(xTest)
numClass = 10
sizeDigit = xTrain.shape[1:]

dimInput = np.prod(xTrain.shape[1:])
xTrain = xTrain.reshape((numTrain, dimInput))
xTest = xTest.reshape((numTest, dimInput))
print(xTrain.shape)
print(xTest.shape)

convVAE = ConvVAE(dimInput, layerDense=layerDense, layerConv=layerConv, ratRecon=ratRecon)
history, timeTrain = convVAE.fit(xTrain, xTest, 
                                 numEpochs=numEpochs,
                                 sizeBatch=sizeBatch,
                                 tempPathBest=modelPath)